<a href="https://colab.research.google.com/github/Catofood/PixelMind/blob/main/Linear_layer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import numpy as np
import math

# глобальные константы
e = math.e

# здесь написаны основные функции активации
def sigmoid(x: np.array):
  return 1/(1 + e ** (-x))


def ReLU(x: np.array):
  return np.maximum(x, 0)


def tanh(x: np.array):
  return (e ** x - e **(-x)) / (e ** x + e ** (-x))


# функции потерь



In [12]:
class Linear:

  def __init__(self, input_size: int, output_size: int, activation_func=None):
    self.n_in = input_size
    self.n_out = output_size
    self.weights = np.random.randn(self.n_in, self.n_out)
    self.bias = np.random.randn(1, self.n_out)

    if activation_func is None:
      self.activation_func = lambda x: x
    else:
      self.activation_func = activation_func


  def forward(self, x: np.array):
     # прямой проход по слою (вычисление значений)
     # y = w * x + b [здесь имеются в виду матричные операции]
     # (x должен быть двумерным: [batch_size, input_size])
     linear_res = np.dot(x, self.weights) + self.bias
     return self.activation_func(linear_res)


In [13]:
class NeuralNetwork:
    """
    Класс нейронной сети, состоящей из нескольких последовательных слоёв.
    """
    def __init__(self, layers=None):
        """
        :param layers: список объектов класса Linear (или пусто)
        """
        if layers is None:
            self.layers = []
        else:
            self.layers = layers


    def add_layer(self, layer: Linear):
        """
        Добавить слой в модель динамически.
        :param layer: объект класса Linear
        """
        self.layers.append(layer)


    def forward(self, x: np.array):
        """
        Прямой проход через всю сеть.
        :param x: входные данные (обычно [batch_size, n_features])
        :return: выходной тензор после последнего слоя
        """
        for layer in self.layers:
            x = layer.forward(x)
        return x


In [14]:
layers = [
        Linear(5, 2, sigmoid),
        Linear(2, 1)
    ]
model = NeuralNetwork(layers=layers)

# Пример входных данных (batch_size=1, размерность входа=5)
x = np.array([[1, 2, 3, -5, -5]])
output = model.forward(x)
print("Результат прямого прохода", output)

Результат прямого прохода [[1.71528089]]
